In [1]:
!pip install -U \
  ctransformers==0.2.5 \
  sentence-transformers==5.0.0 \
  pinecone==7.3.0 \
  langchain_community \
  langchain-huggingface \
  huggingface_hub \
  transformers \
  text-generation \
  langchainhub \
  flask \
  sentencepiece \
  jinja2 \
  bitsandbytes \
  accelerate \
  google-search-results \
  numexpr

  Using cached huggingface_hub-1.3.4-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-5.0.0-py3-none-any.whl.metadata (37 kB)


In [2]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

In [ ]:
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''

#Extracting pdf texts and make chunks


In [4]:
loader = PyPDFLoader('/content/Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

text_chunks = text_splitter.split_documents(docs)

print(f"Total chunks: {len(text_chunks)}")

Total chunks: 5868


#download huggingface embedding model

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
text = 'hello'
print(len(embeddings.embed_query(text))) #dimension of index in pinecone

768


In [7]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_API_ENV"] = PINECONE_API_ENV

In [8]:
texts = [doc.page_content for doc in text_chunks]

# Generate embeddings for all chunks
chunk_embeddings = embeddings.embed_documents(texts)

# Check first embedding
print("Length of first embedding vector:", len(chunk_embeddings[0]))

Length of first embedding vector: 768


In [9]:
pc = Pinecone(
        api_key= PINECONE_API_KEY
    )

index_name = "medicalbot"

dense_index = pc.Index(index_name)


In [10]:
vectorstore = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embeddings,
    index_name=index_name
)

In [11]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

query = 'What are allergies'

docs = docsearch.similarity_search(query, k=2)
docs

[Document(id='c0a864c8-2aee-4e35-815c-b2f29f83b019', metadata={}, page_content='Description\nThe immune system is designed to protect the body\nfrom harmful invaders such as germs. Occasionally, it\ngoes awry and attacks harmless or mildly noxious\nagents, doing more harm than good. This event is termed\nallergy if the target is from the outside—like pollen or\nbee venom—and autoimmunity if it is caused by one of\nthe body’s own components.\nThe immune system usually responds only to certain\nkinds of chemicals, namely proteins. However, non-pro-\nteins can trigger the same sort of response, probably by\naltering a protein to make it look like a target. Physical\nallergy refers to reactions in which a protein is not the\ninitial inciting agent.\nSometimes it takes a combination of elements to\nproduce an allergic reaction. A classic example is drugs'),
 Document(id='5080b5f3-d363-43ff-bfd6-d60441d82324', metadata={}, page_content='KEY TERMS\nAllergen—Any substance that irritates only t

#LLM creation and creating chatbot

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    provider="auto",  # let Hugging Face choose the best provider for you
    huggingfacehub_api_token=''
)

chat_model = ChatHuggingFace(llm=llm)

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a medical information assistant using Retrieval-Augmented Generation (RAG).

STRICT RULES:
- Use ONLY the information in the CONTEXT.
- Do NOT use prior knowledge.
- Do NOT explain your reasoning.
- If the answer is not explicitly stated in the context, reply exactly:
  "I don't have enough information in the provided documents."
- Do NOT provide medical diagnosis or prescriptions.

FORMAT:
only return the helpful answer the below and nothing else.

CONTEXT:
{context}

QUESTION:
{question}

FINAL ANSWER:
""")


In [15]:
from langchain_core.runnables import RunnablePassthrough

# ---- Helper to format retrieved docs ----
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# ---- Retriever ----
retriever = docsearch.as_retriever(search_kwargs={"k": 4})

# ---- RAG Chain ----
chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | chat_model
)

# ---- Chatbot Loop ----
print("🩺 Medical RAG Chatbot")
print("Type your question below.")
print("Type 'exit', 'quit', or 'bye' to stop.\n")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("\nBot: Take care! 👋")
        break

    try:
        response = chain.invoke(user_input)
        print("\nBot:")
        print(response.content)
        print("-" * 50)

    except Exception as e:
        print("\nBot: Sorry, something went wrong.")
        print("Error:", str(e))
        break


🩺 Medical RAG Chatbot
Type your question below.
Type 'exit', 'quit', or 'bye' to stop.

You: symptoms of headace

Bot:
<think>
Hmm, the user is asking about "symptoms of headace" - likely meaning "headache". Looking at the CONTEXT provided, I need to strictly follow the rules: only use information from the context, no prior knowledge, no explanations.

Scanning the context, I see several mentions of headache symptoms:

- Under papilledema symptoms: "headaches, which are usually worse upon awakening and exacerbated by coughing, holding the breath..."
- Later in the context: "headaches can be accompanied by dizziness, nausea, and vomiting" when discussing aneurysms/AVMs
- Also mentioned: "The sudden, severe headache" in relation to subarachnoid hemorrhage

These are all relevant symptoms of headaches extracted directly from the context. The question doesn't specify a type of headache, so I'll include all headache symptoms mentioned in the context without adding interpretations.

I need t